### 時間序列新增  7 日 搭乘量 (7DaysTimes)
### 時間序列次數改為 移動平均 (MATimes)

In [1]:
import time
import pandas as pd
import datetime
import os

In [2]:
# 初始年月份
set_data_year = 2022 #start_year
set_data_month = 12 #start_month
end_year = 2023
end_month = 2

# 檔案位置
Site = './data/'

# 是否輸出檔案 >0 輸出
df_save = 1
# 檔案輸出位置 
custom_output_path = f'./7DaysTimes'
DataName = '202301'

#設定 {N+M+1}DaysTimes 天數 
N = 3 #往前天數
M = 3 #往後天數

#最終留下月份
start_day_str = '2023-01-01'  #含這天
end_day_str   = '2023-02-01'  #不含這天

#-------------------------------------------------------------------------------------------------#
start_day = datetime.datetime.strptime(start_day_str, '%Y-%m-%d')
end_day = datetime.datetime.strptime(end_day_str, '%Y-%m-%d')

# 年月
start_year = set_data_year
start_month = set_data_month

datasite = []
for year in range(start_year, end_year+1):  # 修改为你的年份范围
    if year == start_year:
        if year < end_year :
            for month in range(start_month, 13):  # 生成1到12的月份
                filename = f'{Site}dfTime_TLC-{year}-{month}.parquet'
                datasite.append(filename)            
        elif year == end_year:
            for month in range(start_month, end_month+1):  # 生成1到12的月份
                filename = f'{Site}dfTime_TLC-{year}-{month}.parquet'
                datasite.append(filename)    
    elif year > start_year:
        if year < end_year :
            for month in range(1, 13):  # 生成1到12的月份
                filename = f'{Site}dfTime_TLC-{year}-{month}.parquet'
                datasite.append(filename)
        elif year == end_year:
            for month in range(1, end_month+1):  # 生成1到12的月份
                filename = f'{Site}dfTime_TLC-{year}-{month}.parquet'
                datasite.append(filename)     
                
# 建立一個空的DataFrame來存放合併後的數據
df = pd.DataFrame()

data_year = set_data_year
data_month = set_data_month

# 逐一讀取並合併所有Parquet檔案
for file in datasite:
    # 使用pandas的read_parquet函數讀取Parquet檔案並將其轉換為DataFrame
    df_temp = pd.read_parquet(file)
    # 將數據合併到已有的DataFrame中
    df = pd.concat([df, df_temp], ignore_index=True)
                
print("done時間:", datetime.datetime.now())
df

done時間: 2023-08-27 00:21:48.301860


,Name,year,month,day,hour,weekday,PULocationID,is_holiday,次數
0,lyft,2022,12,1,0,3,3,False,12
1,lyft,2022,12,1,0,3,4,False,23
2,lyft,2022,12,1,0,3,6,False,1
3,lyft,2022,12,1,0,3,7,False,49
4,lyft,2022,12,1,0,3,9,False,3
...,...,...,...,...,...,...,...,...,...
1255054,yellow,2023,2,28,23,1,255,False,1
1255055,yellow,2023,2,28,23,1,256,False,1
1255056,yellow,2023,2,28,23,1,261,False,15
1255057,yellow,2023,2,28,23,1,262,False,10


In [3]:
#檢查用
datasite

['./data/dfTime_TLC-2022-12.parquet',
 './data/dfTime_TLC-2023-1.parquet',
 './data/dfTime_TLC-2023-2.parquet']

### 新增 前N後M日搭乘量 (NDaysTimes)
    預設前3後3 總共7天

In [4]:
# 記錄開始時間
start_time = time.time()

#創立一個 df_temp 統計N天內總次數
df['TempTime'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str)+'-' +df['day'].astype(str),
                                format='%Y-%m-%d')
df_temp = df[['Name','PULocationID','TempTime','次數']]
df_temp = df_temp.groupby([ 'Name','TempTime','PULocationID'])['次數'].sum().reset_index()

TempTime_unique_count = df['TempTime'].nunique()
print(f'總天數:{TempTime_unique_count}')
temp = 0
temp1 = '0'

for index, row in df_temp.iterrows():
    temp_start_time = row['TempTime'] - pd.DateOffset(days=N)
    temp_end_time = row['TempTime'] + pd.DateOffset(days=M)
    temp_count = df_temp.loc[(df_temp['TempTime'] >= temp_start_time) & (df_temp['TempTime'] <= temp_end_time) & (row['Name'] == df_temp['Name']) & (row['PULocationID'] == df_temp['PULocationID']) ]['次數'].sum()
    temp_day =  row['TempTime'] - df_temp['TempTime'].min()
    if row['TempTime'].day == 1:  # 每個月的1日印出來
        if (temp != row['TempTime']) | (temp1  != row['Name']) :
            temp = row['TempTime']
            temp1 = row['Name']
            print(row['Name'],row['TempTime'])
    if TempTime_unique_count - temp_day.days <= M:          
        temp_count =  df_temp.loc[(df_temp['TempTime'] >= df_temp['TempTime'].max() -pd.DateOffset(days=N)-pd.DateOffset(days=M)) & (df_temp['TempTime'] <= df_temp['TempTime'].max() ) & (row['Name'] == df_temp['Name']) & (row['PULocationID'] == df_temp['PULocationID']) ]['次數'].sum()
        df_temp.at[index,f'{N+1+M}DaysTimes'] = temp_count
    elif temp_day.days >= N:  
        df_temp.at[index,f'{N+1+M}DaysTimes'] = temp_count
    else : #處理前N-1天
        temp_count =  df_temp.loc[(df_temp['TempTime'] >= df_temp['TempTime'].min()) & (df_temp['TempTime'] <= ( df_temp['TempTime'].min() +pd.DateOffset(days=N)+pd.DateOffset(days=M)) ) & (row['Name'] == df_temp['Name']) & (row['PULocationID'] == df_temp['PULocationID']) ]['次數'].sum()
        #temp_count = temp_count *(N+1) / (temp_day.days+1)        
        df_temp.at[index,f'{N+1+M}DaysTimes'] = temp_count
        
del temp_start_time,temp_end_time,temp_count,temp_day,temp,temp1  

# 放回 df
#df[f'{N+1+M}DaysTimes'] = df.apply(lambda row:df_temp.loc[(df_temp['Name'] == row['Name']) &(df_temp['TempTime'] == row['TempTime']),f'{N+1+M}DaysTimes'].values[0], axis=1)
merged_df = df.merge(df_temp[['Name', 'TempTime','PULocationID', f'{N+1+M}DaysTimes']], on=['Name', 'TempTime','PULocationID'], how='left')
df[f'{N+1+M}DaysTimes'] = merged_df[f'{N+1+M}DaysTimes']
del merged_df

#留下需求月份
df = df[(df['TempTime'] >= start_day) &  (df['TempTime'] < end_day) ]

# TempTime移除
df = df.drop(columns=['TempTime'])
#df['TempTime'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str)+'-' +df['day'].astype(str) + ' ' + df['hour'].astype(str) ,format='%Y-%m-%d %H')


# 輸出檔案
if df_save > 0 :
    os.makedirs(os.path.dirname(f'{custom_output_path}/dfTime_TLC-{DataName}.parquet'), exist_ok=True)
    df.to_parquet(f'{custom_output_path}/dfTime_TLC-{DataName}.parquet')
    print('save!')
    
# 記錄結束時間
end_time = time.time()   
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

總天數:90
lyft 2022-12-01 00:00:00
lyft 2023-01-01 00:00:00
lyft 2023-02-01 00:00:00
uber 2022-12-01 00:00:00
uber 2023-01-01 00:00:00
uber 2023-02-01 00:00:00
yellow 2022-12-01 00:00:00
yellow 2023-01-01 00:00:00
yellow 2023-02-01 00:00:00
save!

程式執行花費的時間： 426.87 秒
done時間: 2023-08-27 00:28:56.995356


In [12]:
df[df['PULocationID']==3].head()

,Name,year,month,day,hour,weekday,PULocationID,is_holiday,次數,7DaysTimes
434764,lyft,2023,1,1,0,6,3,True,12,2403.0
435013,lyft,2023,1,1,1,6,3,True,28,2403.0
435259,lyft,2023,1,1,2,6,3,True,19,2403.0
435508,lyft,2023,1,1,3,6,3,True,26,2403.0
435754,lyft,2023,1,1,4,6,3,True,17,2403.0


In [18]:
y = 2023
m = 1
d = 31
Loc = 260
df[(df['year']==y ) & (df['month']== m ) & (df['day']== d ) & (df['PULocationID']== Loc )]

,Name,year,month,day,hour,weekday,PULocationID,is_holiday,次數,7DaysTimes
608667,lyft,2023,1,31,0,1,260,False,14,5067.0
608889,lyft,2023,1,31,1,1,260,False,11,5067.0
609104,lyft,2023,1,31,2,1,260,False,4,5067.0
609318,lyft,2023,1,31,3,1,260,False,13,5067.0
609545,lyft,2023,1,31,4,1,260,False,20,5067.0
609777,lyft,2023,1,31,5,1,260,False,21,5067.0
610019,lyft,2023,1,31,6,1,260,False,31,5067.0
610268,lyft,2023,1,31,7,1,260,False,49,5067.0
610516,lyft,2023,1,31,8,1,260,False,49,5067.0
610762,lyft,2023,1,31,9,1,260,False,46,5067.0


In [7]:
#檢查用
df_temp[df_temp['PULocationID']==100].head(10)

,Name,TempTime,PULocationID,次數,7DaysTimes
98,lyft,2022-12-01,100,1025,7499.0
355,lyft,2022-12-02,100,1158,7499.0
611,lyft,2022-12-03,100,1237,7499.0
867,lyft,2022-12-04,100,1048,7499.0
1123,lyft,2022-12-05,100,954,7578.0
1379,lyft,2022-12-06,100,1012,7591.0
1636,lyft,2022-12-07,100,1065,7663.0
1893,lyft,2022-12-08,100,1104,7625.0
2149,lyft,2022-12-09,100,1171,7652.0
2405,lyft,2022-12-10,100,1309,7473.0
